# merging prediction data with manual pixel data

### This Jupyter Notebook is crafted to **analyze** and **compare** measurements of prawn exuviae (molted shells) using data from two distinct sources. The analysis workflow includes merging datasets, cleaning data, and calculating measurement differences to evaluate accuracy and consistency.

## Key Steps:

###  1. **Load Datasets**: Import datasets containing exuviae measurements from various sources.
###  2. **Data Cleaning and Preprocessing**: Remove unnecessary prefixes and categorize exuviae into size classes (big and small).
###  3. **Merge Datasets**: Combine datasets based on common identifiers to enable comparison.
###  4. **Calculate Differences**: Compute the absolute differences between measurements from the two sources.
###  5. **Filter and Analyze**: Examine the data to identify discrepancies and validate measurement accuracy.

##  Dependencies:

# - **pandas**: Utilized for data manipulation and analysis.
# - **numpy**: Employed for numerical operations.


In [1]:
import pandas as pd





df_shai_exuviae=pd.read_csv('/Users/gilbenor/Documents/code_projects/msc/counting_research_algorithms/fifty_one and analysis/measurements/exuviae/spreadsheet_files/Results-shai-exuviae.csv')
df_analysis=pd.read_csv('/Users/gilbenor/Documents/code_projects/msc/counting_research_algorithms/training and val output/runs/pose/predict83/length_analysis_new_split.csv')


# Remove 'colored_' prefix from image_name column
df_analysis['image_name'] = df_analysis['image_name'].str.replace('colored_', '')





In [2]:
# Remove prefix from Label column
df_shai_exuviae['Label'] = df_shai_exuviae['Label'].str.replace('Shai - exuviae:', '')

# Create columns for big and small lengths
df_shai_exuviae['is_big'] = df_shai_exuviae.groupby('Label')['Length'].transform(lambda x: x == x.max() if len(x) > 1 else False)
df_shai_exuviae['is_small'] = df_shai_exuviae.groupby('Label')['Length'].transform(lambda x: x == x.min() if len(x) > 1 else False)

df_shai_exuviae

,,Label,Area,Mean,Min,Max,BX,BY,Width,Height,Angle,Length,is_big,is_small
0,1,undistorted_GX010191_5_190,1030,131.196,29.609,166.129,3660,36,41,1029,-87.772,1028.778,False,False
1,2,undistorted_GX010191_8_309,791,156.202,55.333,175.122,2456,1376,733,297,22.017,789.582,False,True
2,3,undistorted_GX010191_8_309,914,136.620,84.361,170.768,3860,1736,157,901,-80.166,913.420,True,False
3,4,undistorted_GX010191_10_370,697,127.490,40.000,189.921,3092,1716,245,653,69.482,696.161,False,True
4,5,undistorted_GX010191_10_370,940,154.028,81.667,180.810,4084,1360,837,429,-27.111,939.191,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,81,undistorted_GX010194_26_373,1218,150.946,81.667,179.000,1072,1548,1053,613,30.189,1217.065,False,False
81,82,undistorted_GX010194_39_513,1309,156.869,47.333,179.686,2436,1320,33,1309,-91.401,1308.391,False,False
82,83,undistorted_GX010194_60_836,1735,134.509,47.373,170.459,2208,2544,1733,77,177.487,1733.667,True,False
83,84,undistorted_GX010194_60_836,1380,156.280,64.333,181.049,2608,52,561,1261,66.038,1378.840,False,True


In [3]:
# Create a merged dataframe by matching Label with image_name
merged_df = pd.merge(
    df_shai_exuviae,
    df_analysis,
    left_on='Label',
    right_on='image_name',
    how='inner'
)

# Calculate the absolute difference between Length and pixels_total_length
merged_df['length_diff'] = abs(merged_df['Length'] - merged_df['pixels_total_length'])

# Filter for rows where the lengths are close (e.g. within 10% difference)
threshold = 0.1# 10% threshold

# Group by Label to find images with multiple detections
duplicate_labels = merged_df.groupby('Label').filter(lambda x: len(x) > 1)['Label'].unique()

# For images with multiple detections, match the detections with size classes
for label in duplicate_labels:
    # Get rows for this label
    label_rows = merged_df[merged_df['Label'] == label]
    
    # Get the big and small detections from Shai's data
    shai_big = label_rows[label_rows['is_big']]['Length'].values[0] if any(label_rows['is_big']) else None
    shai_small = label_rows[label_rows['is_small']]['Length'].values[0] if any(label_rows['is_small']) else None
    
    # Get the big and small detections from our measurements
    our_big = label_rows[label_rows['lobster_size'] == 'big']['pixels_total_length'].values[0] if any(label_rows['lobster_size'] == 'big') else None
    our_small = label_rows[label_rows['lobster_size'] == 'small']['pixels_total_length'].values[0] if any(label_rows['lobster_size'] == 'small') else None
    
    # Update is_big and is_small flags based on our size classes
    merged_df.loc[merged_df['Label'] == label, 'is_big'] = merged_df.loc[merged_df['Label'] == label, 'lobster_size'] == 'big'
    merged_df.loc[merged_df['Label'] == label, 'is_small'] = merged_df.loc[merged_df['Label'] == label, 'lobster_size'] == 'small'

# Recalculate length difference after matching
merged_df['length_diff'] = abs(merged_df['Length'] - merged_df['pixels_total_length'])



close_matches = merged_df[merged_df['length_diff'] / merged_df['Length'] < threshold]

# Sort by length difference to see best matches first
close_matches = close_matches.sort_values('length_diff')

# Display results
print(f"Found {len(close_matches)} matches where lengths are within {threshold*100}% difference")
display(close_matches)


Found 51 matches where lengths are within 10.0% difference


,,Label,Area,Mean,Min,Max,BX,BY,Width,Height,...,is_small,image_name,lobster_size,total_length,carapace_length,eye_x,eye_y,pixels_total_length,pixels_carapace_length,length_diff
48,31,undistorted_GX010191_37_1242,720,148.253,98.659,170.629,2764,948,221,685,...,True,undistorted_GX010191_37_1242,small,139.2,34.5,2927.6,1243.5,719.0,178.1,0.490
2,2,undistorted_GX010191_8_309,791,156.202,55.333,175.122,2456,1376,733,297,...,True,undistorted_GX010191_8_309,small,154.8,35.7,2879.4,1465.9,790.1,181.9,0.518
24,19,undistorted_GX010191_32_305,791,147.524,87.960,176.063,2516,1328,741,277,...,True,undistorted_GX010191_32_305,small,154.2,36.9,2897.1,1454.2,786.3,187.9,3.495
97,67,undistorted_GX010193_11_1065,1742,130.953,78.167,171.020,2152,1256,973,1445,...,False,undistorted_GX010193_11_1065,big,184.3,44.3,2701.3,2017.7,1745.4,419.2,4.733
75,50,undistorted_GX010191_100_1250,734,145.590,35.000,185.270,2776,1088,137,721,...,True,undistorted_GX010191_100_1250,small,143.0,38.0,2853.8,1416.3,739.4,196.5,6.668
102,72,undistorted_GX010193_27_1553,1767,128.617,46.667,162.577,1672,108,253,1749,...,False,undistorted_GX010193_27_1553,big,186.6,47.3,1853.2,762.9,1774.1,449.8,8.029
114,82,undistorted_GX010194_39_513,1309,156.869,47.333,179.686,2436,1320,33,1309,...,False,undistorted_GX010194_39_513,small,144.1,33.7,2407.7,2016.2,1300.3,303.9,8.091
70,48,undistorted_GX010191_94_1132,823,153.479,28.338,180.844,3052,368,777,273,...,False,undistorted_GX010191_94_1132,big,156.7,34.1,3728.6,1885.2,832.0,181.0,9.711
40,27,undistorted_GX010191_35_1167,771,154.688,47.898,184.515,2844,644,761,125,...,True,undistorted_GX010191_35_1167,small,149.2,34.8,3262.8,683.9,759.4,177.2,10.649
22,18,undistorted_GX010191_31_283,947,138.038,53.297,171.404,3776,1492,105,941,...,False,undistorted_GX010191_31_283,big,174.8,37.3,3796.4,1995.6,931.8,198.7,13.936


In [4]:
len(close_matches['Label'].unique())

39

In [5]:
# Select only the specified columns
selected_columns = ['image_name', 'BX', 'BY', 'Width', 'Height', 'Length', 
                   'lobster_size', 'total_length', 'pixels_total_length', 'length_diff','is_big','is_small']

new_df = close_matches[selected_columns]

# Display filtered results
display(new_df)


,image_name,BX,BY,Width,Height,Length,lobster_size,total_length,pixels_total_length,length_diff,is_big,is_small
48,undistorted_GX010191_37_1242,2764,948,221,685,718.510,small,139.2,719.0,0.490,False,True
2,undistorted_GX010191_8_309,2456,1376,733,297,789.582,small,154.8,790.1,0.518,False,True
24,undistorted_GX010191_32_305,2516,1328,741,277,789.795,small,154.2,786.3,3.495,False,True
97,undistorted_GX010193_11_1065,2152,1256,973,1445,1740.667,big,184.3,1745.4,4.733,False,False
75,undistorted_GX010191_100_1250,2776,1088,137,721,732.732,small,143.0,739.4,6.668,False,True
102,undistorted_GX010193_27_1553,1672,108,253,1749,1766.071,big,186.6,1774.1,8.029,True,False
114,undistorted_GX010194_39_513,2436,1320,33,1309,1308.391,small,144.1,1300.3,8.091,False,False
70,undistorted_GX010191_94_1132,3052,368,777,273,822.289,big,156.7,832.0,9.711,True,False
40,undistorted_GX010191_35_1167,2844,644,761,125,770.049,small,149.2,759.4,10.649,False,True
22,undistorted_GX010191_31_283,3776,1492,105,941,945.736,big,174.8,931.8,13.936,True,False


In [6]:
# Add real length based on size class
new_df['real_length'] = new_df['lobster_size'].map({'small': 145, 'big': 180})

# Calculate absolute difference between real length and total length
new_df['real_length_abs_diff'] = abs(new_df['total_length'] - new_df['real_length'])

# Calculate relative difference as percentage
new_df['real_length_rel_diff'] = (new_df['real_length_abs_diff'] / new_df['real_length']) * 100

display(new_df)


/var/folders/9w/y40_ws0n1_n_hby5lpbrj1_m0000gn/T/ipykernel_68021/2314326851.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['real_length'] = new_df['lobster_size'].map({'small': 145, 'big': 180})
/var/folders/9w/y40_ws0n1_n_hby5lpbrj1_m0000gn/T/ipykernel_68021/2314326851.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['real_length_abs_diff'] = abs(new_df['total_length'] - new_df['real_length'])
/var/folders/9w/y40_ws0n1_n_hby5lpbrj1_m0000gn/T/ipykernel_68021/2314326851.py:8: SettingWit

,image_name,BX,BY,Width,Height,Length,lobster_size,total_length,pixels_total_length,length_diff,is_big,is_small,real_length,real_length_abs_diff,real_length_rel_diff
48,undistorted_GX010191_37_1242,2764,948,221,685,718.510,small,139.2,719.0,0.490,False,True,145,5.8,4.000000
2,undistorted_GX010191_8_309,2456,1376,733,297,789.582,small,154.8,790.1,0.518,False,True,145,9.8,6.758621
24,undistorted_GX010191_32_305,2516,1328,741,277,789.795,small,154.2,786.3,3.495,False,True,145,9.2,6.344828
97,undistorted_GX010193_11_1065,2152,1256,973,1445,1740.667,big,184.3,1745.4,4.733,False,False,180,4.3,2.388889
75,undistorted_GX010191_100_1250,2776,1088,137,721,732.732,small,143.0,739.4,6.668,False,True,145,2.0,1.379310
102,undistorted_GX010193_27_1553,1672,108,253,1749,1766.071,big,186.6,1774.1,8.029,True,False,180,6.6,3.666667
114,undistorted_GX010194_39_513,2436,1320,33,1309,1308.391,small,144.1,1300.3,8.091,False,False,145,0.9,0.620690
70,undistorted_GX010191_94_1132,3052,368,777,273,822.289,big,156.7,832.0,9.711,True,False,180,23.3,12.944444
40,undistorted_GX010191_35_1167,2844,644,761,125,770.049,small,149.2,759.4,10.649,False,True,145,4.2,2.896552
22,undistorted_GX010191_31_283,3776,1492,105,941,945.736,big,174.8,931.8,13.936,True,False,180,5.2,2.888889


In [7]:
# Calculate absolute difference between pixels_total_length and Length
new_df['pixel_abs_diff'] = abs(new_df['pixels_total_length'] - new_df['Length'])

# Calculate relative difference as percentage 
new_df['pixel_rel_diff'] = (new_df['pixel_abs_diff'] / new_df['Length']) * 100

display(new_df)

print(len(new_df))


/var/folders/9w/y40_ws0n1_n_hby5lpbrj1_m0000gn/T/ipykernel_68021/4111279222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['pixel_abs_diff'] = abs(new_df['pixels_total_length'] - new_df['Length'])
/var/folders/9w/y40_ws0n1_n_hby5lpbrj1_m0000gn/T/ipykernel_68021/4111279222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['pixel_rel_diff'] = (new_df['pixel_abs_diff'] / new_df['Length']) * 100


,image_name,BX,BY,Width,Height,Length,lobster_size,total_length,pixels_total_length,length_diff,is_big,is_small,real_length,real_length_abs_diff,real_length_rel_diff,pixel_abs_diff,pixel_rel_diff
48,undistorted_GX010191_37_1242,2764,948,221,685,718.510,small,139.2,719.0,0.490,False,True,145,5.8,4.000000,0.490,0.068197
2,undistorted_GX010191_8_309,2456,1376,733,297,789.582,small,154.8,790.1,0.518,False,True,145,9.8,6.758621,0.518,0.065604
24,undistorted_GX010191_32_305,2516,1328,741,277,789.795,small,154.2,786.3,3.495,False,True,145,9.2,6.344828,3.495,0.442520
97,undistorted_GX010193_11_1065,2152,1256,973,1445,1740.667,big,184.3,1745.4,4.733,False,False,180,4.3,2.388889,4.733,0.271907
75,undistorted_GX010191_100_1250,2776,1088,137,721,732.732,small,143.0,739.4,6.668,False,True,145,2.0,1.379310,6.668,0.910019
102,undistorted_GX010193_27_1553,1672,108,253,1749,1766.071,big,186.6,1774.1,8.029,True,False,180,6.6,3.666667,8.029,0.454625
114,undistorted_GX010194_39_513,2436,1320,33,1309,1308.391,small,144.1,1300.3,8.091,False,False,145,0.9,0.620690,8.091,0.618393
70,undistorted_GX010191_94_1132,3052,368,777,273,822.289,big,156.7,832.0,9.711,True,False,180,23.3,12.944444,9.711,1.180972
40,undistorted_GX010191_35_1167,2844,644,761,125,770.049,small,149.2,759.4,10.649,False,True,145,4.2,2.896552,10.649,1.382899
22,undistorted_GX010191_31_283,3776,1492,105,941,945.736,big,174.8,931.8,13.936,True,False,180,5.2,2.888889,13.936,1.473561


51


In [8]:
new_df.to_csv('/Users/gilbenor/Documents/code_projects/msc/counting_research_algorithms/fifty_one and analysis/measurements/exuviae/spreadsheet_files/length_analysis_new_split_shai_exuviae.csv', index=False)